<a href="https://colab.research.google.com/github/martinopiaggi/summarize/blob/main/Martino_Summarize_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#@markdown # **Source of the summary**

Type = "Youtube video or playlist" #@param ['Text', 'Text from Google Drive','Youtube video or playlist', 'Videos on Google Drive folder']
#@markdown ---
#@markdown #### **Text (only if Type is Text)**
Text = "Making your bed in the morning, that first cup of coffee, grabbing a croissant on your way to work, listening to your favorite podcast on the train\u2026 As much as we wish for each day to be different, repeating some of the same actions is an important part of our lives.  Researchers have found that no more than 40% of our actions are consciously self-selected. Instead, we perform these actions in an automated way, without conscious awareness. How can you ramp up that percentage and live a more intentional life?  The key is to understand the difference between habits, routines, and rituals, and to design a life where your daily actions allow you to play with the entire spectrum of consciousness.  Shades of Consciousness Waking up, commuting, walking past a particular store, or starting a meeting at work are common cues that can trigger actions as diverse as smoking a cigarette, buying a croissant, or drinking coffee.  Many books have been written about building better habits and breaking bad ones. The most famous is probably Atomic Habits by James Clear (2018), but other ones such as The Seven Habits of Highly Effective People by Stephen Covey (1989) and The Power of Habits by Charles Duhigg (2012) have also sold millions of copies. It\u2019s fair to say that people are convinced habits matter.  And it makes sense. To maintain a healthy lifestyle, it helps to be able to set some behaviors on autopilot so that you don\u2019t have to make a conscious effort every single time. Habits are great for those actions.  But most good habits don\u2019t start as habits. They start as routines. The main difference between habits and routines is how aware and intentional you are. A habit usually manifests itself as an automatic urge to do something, often triggered by a particular cue. The stronger the connection between the trigger and the habit, the more ingrained the habit.  In contrast, routines require deliberate practice. Making your bed in the morning, going to the gym, going for a hike every Sunday, and meditating are all routines that require you to keep consciously practicing them or they eventually die out. Your brain will not go into automatic mode and walk you to the gym for your weekly HIIT class.  Both habits and routines are regular and repeated actions, but habits happen with little or no conscious thought, whereas routines require a higher degree of intention and effort.  With enough time, routines can turn into habits, but you need enough repetitions to create that habit loop:  Cue. Choose a trigger to tell your brain to start the routine you want to turn into a habit. Routine. Execute the routine, ideally starting with a small, actionable chunk. Reward. Do something enjoyable to tell your brain that this particular action is worth performing again in the future. When it comes to healthy habits, automaticity is good. Building a loop is good. But what about the actions where you actually want to make a conscious effort, the ones where you get satisfaction from pushing yourself out of your comfort zone?  How can you nurture and maintain more intentionality \u2014 in short, break the loop \u2014 with these actions?  From Routine to Ritual We tend to associate rituals with very specific types of activities: communal rites of worship, rites of passage, commemorative rites\u2026 Yes, these are rituals, but this is only the narrowest definition of the term.  More broadly, the difference between a routine and a ritual is the mindset behind the action. While routines can be actions that just need to be done\u2014such as making your bed or taking a shower\u2014rituals are viewed as more meaningful practices that have a real sense of purpose.  Design your habits, routines, and rituals with the intentionality curve Rituals do not have to be spiritual or religious. What matters is your level of intentionality. With rituals, you are fully engaged with a focus on the experience of the task, rather than its mere completion. You are investing your highest levels of energy and consciousness.  And you can virtually turn any routine into a ritual by becoming more mindful and making mental space for the action. For instance, when you eat, you could practice paying attention to the textures and the way you chew. Research actually shows that mindful eating can indeed improve the flavor of your food, making you feel more satisfied.  Showering can become an opportunity to become mindful of your body and its connection to your mind. Focus on the sensation of the water on your skin and how your thoughts seem to flow more easily. This way, a simple morning routine can become a morning ritual.  Even cleaning the house can be used to become more aware of your body movements and sensations in your muscles and joints. Just look at some of your existing routines and see if any could become more intentional.  The Intentional Life The power of playing with the spectrum of consciousness when performing daily activities is that you don\u2019t need to carve extra time for a separate mindfulness practice. Yes, there is lots of research showing the benefits of journaling, yoga, and meditation, but sometimes life gets busy.  Turning a daily routine into a daily ritual is an easy way to inject more intentionality into your life, even when you don\u2019t have much time or energy.  And being aware of your consciousness levels can also help you create better habits. I call this intentional process of scaling up or down your consciousness levels when performing daily actions the Intentionality Curve. Just ask yourself:  What routines do I want to turn into habits by lowering my intentionality? What routines do I want to turn into rituals by increasing my intentionality? Those two simple questions, if you ask them regularly, can help you practice metacognition in a very tangible way and avoid living your life on autopilot. And that\u2019s an idea worth playing with." #@param {type:"string"}
#@markdown ---
#@markdown #### **Youtube video or playlist**
URL = "https://youtu.be/L_Guz73e6fw" #@param {type:"string"}
#@markdown ---
#@markdown #### **Google Drive video, audio (mp4, wav), or folder containing video and/or audio files**
video_path = "Colab Notebooks/transcription/my_video.mp4" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the source**



Using https://huggingface.co/facebook/bart-large-cnn

In [25]:
#@markdown ---
#@markdown # **Install libraries**
#@markdown This cell will take a little while to download several libraries

#@markdown ---
!pip install transformers
!pip install tensorflow
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=0)

import re
import math

if Type == ("Youtube video or playlist" or 'Videos on Google Drive folder'):

  video_path_local_list = []
  !pip install git+https://github.com/openai/whisper.git
  !sudo apt update && sudo apt install ffmpeg

if Type == "Youtube video or playlist":
  !pip install yt-dlp
  from pathlib import Path
  import yt_dlp

  ydl_opts = {
  'format': 'm4a/bestaudio/best',
  'outtmpl': '%(id)s.%(ext)s',
  # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
  'postprocessors': [{  # Extract audio using ffmpeg
  'key': 'FFmpegExtractAudio',
  'preferredcodec': 'wav',
  }]
  }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download([URL])
    list_video_info = [ydl.extract_info(URL, download=False)]

  for video_info in list_video_info:
    video_path_local_list.append(Path(f"{video_info['id']}.wav"))

  for video_path_local in video_path_local_list:
    if video_path_local.suffix == ".mp4":
        video_path_local = video_path_local.with_suffix(".wav")
    result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-9e3f_4sb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-9e3f_4sb
  Resolved https://github.com/openai/whisper.git to commit b38a1f20f4b23f3f3099af2c3e0ca95627276ddf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [555 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,130 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,398 kB]
Hit:13 https://pp

NameError: ignored

In [ ]:
args = dict(
    language = None,
    verbose = 'Live transcription',
    task = 'transcribe'
    temperature = 0.15
    temperature_increment_on_fallback = 0.2
    best_of = 5
    beam_size = 8
    patience = 1.0
    length_penalty = -0.05
    suppress_tokens = "-1"
    initial_prompt = ""
    condition_on_previous_text = True
    fp16 = True
    compression_ratio_threshold = 2.4
    logprob_threshold = -1.0
    no_speech_threshold = 0.6
)


for video_path_local in video_path_local_list:
    display(Markdown(f"### {video_path_local}"))

    video_transcription = !whisper "video_path_local" --model small
    print(video_transcription)


### L_Guz73e6fw.wav

[00:00.000 --> 00:03.840]  We have been a misunderstood and badly mocked org for a long time.
[00:03.840 --> 00:09.600]  Like when we started, we announced the org at the end of 2015,
[00:10.800 --> 00:12.320]  and said we were going to work on AGI.
[00:12.320 --> 00:14.320]  Like people thought we were batshit insane.
[00:16.720 --> 00:23.760]  I remember at the time, a eminent AI scientist at a large industrial AI lab
[00:24.320 --> 00:29.600]  was like DMing individual reporters being like, these people aren't very good,
[00:29.600 --> 00:33.040]  and it's ridiculous to talk about AGI, and I can't believe you're giving them time of day,
[00:33.040 --> 00:38.160]  and it's like that was the level of like pettiness and rancor in the field that a new group of people
[00:38.160 --> 00:43.600]  say we're going to try to build AGI. So open AI and DeepMind was a small collection of folks
[00:43.600 --> 00:52.160]  who were brave enough to talk about AGI in the face of mockery. We don't get

KeyboardInterrupt: ignored

In [20]:

summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=0)

Text = re.sub(r'\n|\r', ' ', Text)
Text = re.sub(r' +', ' ', Text)
Text = Text.strip()

tokenizer = summarizer.tokenizer
tokens = tokenizer.encode(Text)

print(len(tokens))


# Calculate the number of chunks needed
chunk_len = math.ceil(len(tokens) / 512)
chunksNumber = len(tokens)//chunk_len

# Split the tokens into chunks
chunks = [tokens[i:i+chunksNumber] for i in range(0, len(tokens), chunksNumber)]

if(len(chunks)>1):
  if (len(chunks[-1]) + len(chunks[-2])) < 1024:
      merged_chunk = chunks.pop(-1) + chunks.pop(-1)
      chunks.append(merged_chunk)

summary = ''

for chunk in chunks:



    #print(len(chunk))
    # Set max_length and min_length based on token count
    max_length = len(chunk) // 2
    min_length = len(chunk) // 5

    #Generate summary for each chunk without sampling (example)
    summary_chunk = summarizer(tokenizer.decode(chunk), max_length=max_length, min_length=min_length, do_sample=True)
    summary += summary_chunk[0]['summary_text'] + "\n"

print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (1179 > 1024). Running this sequence through the model will result in indexing errors


1179
Researchers have found that no more than 40% of our actions are consciously self-selected. To maintain a healthy lifestyle, it helps to be able to set some behaviors on autopilot so that you don’t have to make a conscious effort every single time. The key is to understand the difference between habits, routines, and rituals, and to design a life where your daily actions allow you to play with the entire spectrum of consciousness.
Turning a daily routine into a daily ritual is an easy way to inject more intentionality into your life. Making your bed in the morning, going to the gym, going for a hike every Sunday, and meditating are all routines that require you to keep consciously practicing them. Rituals do not have to be spiritual or religious. What matters is your level of intentionality. With rituals, you are fully engaged with a focus.  on the experience of the task, rather than its mere completion. The power of playing with the spectrum of consciousness when performing daily 